In [3]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

2022-06-03 09:47:02.890260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-03 09:47:02.890341: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
DATASET = './datasets/headlines_6_1_2022.csv'


In [5]:
df = pd.read_csv(DATASET)
df.head()

,title,date,text,authors,source,url,image_url,search_term
0,Elon Musk tells Tesla staff: return to office ...,2022-06-01T18:37:00Z,June 1 (Reuters) - Tesla Inc (TSLA.O) Chief Ex...,NaN,Reuters,https://www.reuters.com/technology/musk-memo-t...,https://www.reuters.com/resizer/yMbKv1SoP3YjBq...,headlines
1,Mortgage rates rise sharply after three weeks ...,2022-06-01T18:16:34Z,"Mortgage rates rose sharply this week, after p...",Diana Olick,CNBC,https://www.cnbc.com/2022/06/01/mortgage-rates...,https://image.cnbcfm.com/api/v1/image/10706971...,headlines
2,United Airlines plans $100 million expansion o...,2022-06-01T18:00:01Z,United Airlines plans to break ground Wednesda...,Leslie Josephs,CNBC,https://www.cnbc.com/2022/06/01/united-airline...,https://image.cnbcfm.com/api/v1/image/10637412...,headlines
3,Shell's North Sea Jackdaw gas field gets final...,2022-06-01T17:26:15Z,Shell's North Sea Jackdaw gas field has receiv...,"Sharon Marris, news reporter",Sky.com,https://news.sky.com/story/shells-north-sea-ja...,https://e3.365dm.com/22/06/1600x900/skynews-sh...,headlines
4,Frasers Group buys Missguided out of administr...,2022-06-01T17:22:00Z,Show caption Mike Ashley’s Frasers Group owns ...,Joanna Partridge,The Guardian,https://amp.theguardian.com/business/2022/jun/...,NaN,headlines


In [20]:
def clean_text(text):
    final = list() 
    for line in text.split("\n"):
        if line.startswith('Register now for FREE'):
            continue

        if line == "":
            continue
    
        final.append(line)

    return "\n".join(final)

text = '''
"New York (CNN Business) Jamie Dimon is no meteorologist, but the JPMorgan Chase CEO is predicting an economic ""hurricane"" caused by the war in Ukraine, rising inflation pressures and interest rate hikes from the Federal Reserve.

""Right now it's kind of sunny, things are doing fine. Everyone thinks the Fed can handle this,"" Dimon said at a Bernstein conference. ""That hurricane is right out there down the road coming our way.""

""We just don't know if it's a minor one or Superstorm Sandy. You better brace yourself,"" Dimon said, adding thatis preparing for a ""non-benign environment"" and ""bad outcomes.""

Dimon said that the economy is ""distorted"" by inflation . He's also worried that the Fed is starting to unwind its bond portfolio, a process known as quantitative tightening, at the same time it is raising interest rates. That's something that the market is not prepared for, Dimon said, adding that people will be ""writing about [this] in history books for 50 years,""

But the Fed is in a bind. Dimon said the central bank must raise rates because of surging housing prices and other inflation pressures. He stressed that he still thinks the US banking system is in ""great shape"" and can withstand these challenges.

Dimon also said that JPMorgan Chase is going to do all it can to attract talent to stay on top of the financial world. The CEO said the bank will be ""religious"" about paying well to keep its best workers.

Dimon's more cautious outlook comes just a few days after he sounded a little more upbeat about what's next for the markets and the economy.

Speaking at an analyst meeting at the end of May, Dimon said that there were ""big storm clouds"" on the horizon for the economy but expressed hope that they may ""dissipate.""

""If it was a hurricane, I would tell you that,"" Dimon said at the analyst meeting, adding that current conditions also aren't like the ""tsunami"" that banks faced in 2007 and 2008 when the mortgage market was melting down and several large financial institutions collapsed.

JUST WATCHED Jamie Dimon predicts economic 'hurricane' for US. Here's what a Schwab strategist has to say Replay More Videos ... MUST WATCH Jamie Dimon predicts economic 'hurricane' for US. Here's what a Schwab strategist has to say 03:01

Dimon may not be predicting a tsunami just yet. But a hurricane is bad enough, and certainly more damaging than a regular storm. Dimon said he is also worried about the conflict in Ukraine and the impact it will have on oil prices, predicting on Wednesday that it's in the cards for crude prices to eventually spike as high as $150 to $175 a barrel.

""Wars go bad. They go south. They have unintended consequences,"" he said, adding that this conflict will continue to roil the commodity markets around the world, impacting the prices of oil, gas and wheat."

'''

text = clean_text(text)
print(text)
text = text.strip()

"New York (CNN Business) Jamie Dimon is no meteorologist, but the JPMorgan Chase CEO is predicting an economic ""hurricane"" caused by the war in Ukraine, rising inflation pressures and interest rate hikes from the Federal Reserve.
""Right now it's kind of sunny, things are doing fine. Everyone thinks the Fed can handle this,"" Dimon said at a Bernstein conference. ""That hurricane is right out there down the road coming our way.""
""We just don't know if it's a minor one or Superstorm Sandy. You better brace yourself,"" Dimon said, adding thatis preparing for a ""non-benign environment"" and ""bad outcomes.""
Dimon said that the economy is ""distorted"" by inflation . He's also worried that the Fed is starting to unwind its bond portfolio, a process known as quantitative tightening, at the same time it is raising interest rates. That's something that the market is not prepared for, Dimon said, adding that people will be ""writing about [this] in history books for 50 years,""
But the F

In [38]:
# This is a sample method but it doesn't understand the context 

def summarize(text, per):
    nlp = spacy.load('en_core_web_lg')
    doc = nlp(text)

    tokens = [token for token in doc]
    word_count = Counter()

    for token in tokens:
        word = token.text.strip()

        if word == '”':
            continue
        if word.lower() not in STOP_WORDS and word.lower() not in punctuation:
            word_count[word.lower()] += 1

    max_freq = max(word_count.values())
    #Normalize frequencies

    for word in word_count.keys():
        word_count[word] = word_count[word] / max_freq
    
    sentence_tokens = [sent for sent in doc.sents]

    # Count the score for each sentence
    sentence_scores = Counter()

    for sent in sentence_tokens:
        sent_text = sent.text.strip()
        if len(sent) < 10:
            continue
        for word in sent:
            if word.text.lower() in word_count.keys():
                sentence_scores[sent_text] += word_count[word.text.lower()]


    # print(sentence_scores)
    n = int(len(sentence_tokens) * per)
    highscore_sentences = nlargest(n, sentence_scores, key = sentence_scores.get)
    final = [sentence.strip() for sentence in highscore_sentences if len(sentence) > 10]
    
    return '\n'.join(final)

print("Summayr")
print(summarize(text, 0.2))

Summayr
Speaking at an analyst meeting at the end of May, Dimon said that there were ""big storm clouds"" on the horizon for the economy but expressed hope that they may ""dissipate.""
""If it was a hurricane, I would tell you that,"" Dimon said at the analyst meeting, adding that current conditions also aren't like the ""tsunami"" that banks faced in 2007 and 2008 when the mortgage market was melting down and several large financial institutions collapsed.
You better brace yourself,"" Dimon said, adding thatis preparing for a ""non-benign environment"" and ""bad outcomes.""
Dimon said that the economy is ""distorted"" by inflation .
Jamie Dimon is no meteorologist, but the JPMorgan Chase CEO is predicting an economic ""hurricane"" caused by the war in Ukraine, rising inflation pressures and interest rate hikes from the Federal Reserve.
""Right now it's kind of sunny, things are doing fine.
Dimon said he is also worried about the conflict in Ukraine and the impact it will have on oil p

In [57]:
# Use deep learning approach

# Later 